In [43]:
from Bio import SeqIO, SeqFeature
import pandas as pd
import sys
import os

import typing as t
from Bio.SeqRecord import SeqRecord
import glob
import gzip
import re
from collections import Counter
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import pandas as pd
import tqdm



In [44]:
os.chdir("/home/is6/work/cas_thesis/new_domains")

In [45]:

#header_list = ["IDs", "Components"]
df = pd.read_csv("/home/ilia/projects/misc/sedreh/domains_pfam/assigned_components_annotated_filt.tsv", sep='\t')

In [50]:
df.head()

(1939, 3)

In [47]:
annotation_paths = glob.glob('/mnt/nas/storage/ilia/bacteria/ncbi-genomes-2019-12-29/*.gbff.gz')
def load_file(ID):
    file = [x for x in annotation_paths if ID in x]
    if len(file) > 0:
        return ID, file[0]
    return ''

In [48]:
IDs = df[df.columns[0]]
paths = IDs.apply(load_file).tolist()
paths

[('GCF_000006605.1',
  '/mnt/nas/storage/ilia/bacteria/ncbi-genomes-2019-12-29/GCF_000006605.1_ASM660v1_genomic.gbff.gz'),
 ('GCF_000006985.1',
  '/mnt/nas/storage/ilia/bacteria/ncbi-genomes-2019-12-29/GCF_000006985.1_ASM698v1_genomic.gbff.gz'),
 ('GCF_000007085.1',
  '/mnt/nas/storage/ilia/bacteria/ncbi-genomes-2019-12-29/GCF_000007085.1_ASM708v1_genomic.gbff.gz'),
 ('GCF_000007425.1',
  '/mnt/nas/storage/ilia/bacteria/ncbi-genomes-2019-12-29/GCF_000007425.1_ASM742v1_genomic.gbff.gz'),
 ('GCF_000007465.2',
  '/mnt/nas/storage/ilia/bacteria/ncbi-genomes-2019-12-29/GCF_000007465.2_ASM746v2_genomic.gbff.gz'),
 ('GCF_000007585.1',
  '/mnt/nas/storage/ilia/bacteria/ncbi-genomes-2019-12-29/GCF_000007585.1_ASM758v1_genomic.gbff.gz'),
 ('GCF_000007625.1',
  '/mnt/nas/storage/ilia/bacteria/ncbi-genomes-2019-12-29/GCF_000007625.1_ASM762v1_genomic.gbff.gz'),
 ('GCF_000007705.1',
  '/mnt/nas/storage/ilia/bacteria/ncbi-genomes-2019-12-29/GCF_000007705.1_ASM770v1_genomic.gbff.gz'),
 ('GCF_000007745

In [10]:
def parse_gzipped_gb(path: str) -> t.List[SeqRecord]:
    with gzip.open(path, 'rt') as buffer:
        return list(SeqIO.parse(buffer, 'genbank'))
    

In [11]:
path_split = paths[0][1].split('/')
path_split[len(path_split)-1].replace('.gbff.gz','')

'GCF_000006605.1_ASM660v1_genomic'

In [77]:
from Bio import SeqIO, SeqFeature
import sys
import random

def get_16s_table_row(sample_id, path):
    rows = []
    gbank=parse_gzipped_gb(path)
    
    path_split = path.split('/')
    accession_number = path_split[len(path_split)-1].replace('.gbff.gz','')

    for genome in list(gbank):
        for gene in genome.features:
            if gene.type=="rRNA": 
                if 'product' in gene.qualifiers:
                    if '16S' in gene.qualifiers['product'][0]:
                        start = gene.location.nofuzzy_start
                        end = gene.location.nofuzzy_end
                        if 'db_xref' in gene.qualifiers:
                            gi=[]
                            gi=str(gene.qualifiers['db_xref'])
                            gi=gi.split(":")[1]
                            gi=gi.split("'")[0]
                            rows.append([sample_id,accession_number,gi,genome.description,genome.seq[start:end]])
                        else:
                            rows.append([sample_id,accession_number,None,genome.description,genome.seq[start:end]])
                            
    if len(rows) > 0:
        return random.choice(rows)
    return None

data = [x for x in [get_16s_table_row(path[0], path[1]) for path in paths] if x is not None]
data = pd.DataFrame(data, columns=['sample_id','accession_number','sequence_id','description','sequence'])
data.head()

,sample_id,accession_number,sequence_id,description,sequence
0,GCF_000006605.1,GCF_000006605.1_ASM660v1_genomic,3432443,Corynebacterium jeikeium K411 complete genome,"(T, T, T, T, A, T, G, G, A, G, A, G, T, T, T, ..."
1,GCF_000006985.1,GCF_000006985.1_ASM698v1_genomic,1007411,"Chlorobium tepidum TLS chromosome, complete ge...","(G, T, C, C, C, A, G, T, T, A, C, C, A, G, T, ..."
2,GCF_000007085.1,GCF_000007085.1_ASM708v1_genomic,None,Caldanaerobacter subterraneus subsp. tengconge...,"(C, T, G, G, A, G, A, G, T, T, T, G, A, T, C, ..."
3,GCF_000007425.1,GCF_000007425.1_ASM742v1_genomic,None,"Streptococcus pyogenes MGAS315, complete genome","(T, T, A, A, A, G, A, G, A, G, T, T, T, G, A, ..."
4,GCF_000007465.2,GCF_000007465.2_ASM746v2_genomic,2886073,"Streptococcus mutans UA159 chromosome, complet...","(A, G, A, G, T, T, T, G, A, T, C, C, T, G, G, ..."


In [94]:
data.to_csv('/home/is6/work/cas_thesis/16s_extraction/final_round/thesis_results/unique_data.csv', index=False)
f = open("/home/is6/work/cas_thesis/16s_extraction/final_round/thesis_results/unique_data.fasta", "w")
for index, item in data.iterrows():
    f.write('>' + item.accession_number + (('_' + item.sequence_id) if item.sequence_id is not None else '') + ' ' + item.description + '\n' + str(item.sequence) + '\n')
f.close()

In [69]:
#finds the duplicated sequences and joins theirs ids together

from Bio import SeqIO
from collections import defaultdict

dedup_records = defaultdict(list)
for record in SeqIO.parse("/home/is6/work/cas_thesis/16s_extraction/final_round/thesis_results/16S_sequence_final.fasta", "fasta"):
    # Use the sequence as the key and then have a list of id's as the value
    dedup_records[str(record.seq)].append(record.id)
with open("/home/is6/work/cas_thesis/16s_extraction/final_round/thesis_results/duplicate_sequences.fasta", 'w') as output:
    for seq, ids in dedup_records.items():
        # Join the ids and write them out as the fasta
        output.write(">{}\n".format('|'.join(ids)))
        output.write(seq + "\n")

In [72]:
#group_ids = pd.DataFrame(data.groupby(['sample_id'], as_index=False, sort=False).agg(' '.join))
#group_ids[group_ids.sample_id == 'GCF_000006985.1']


1933

In [ ]:
#import numpy as np
#df = data.drop_duplicates(subset='accession_number')
#np.unique(df.accession_number.values).shape

In [52]:
#from Bio import SeqIO

ids = [seq.id for seq in SeqIO.parse('/home/is6/work/cas_thesis/16s_extraction/final_round/thesis_results/16S_sequence_final.fasta', 'fasta')]
len(ids)

8665

In [55]:
data.shape

(1933, 5)

In [ ]:
#import numpy as np
#df = data.drop_duplicates(subset='accession_number')
#np.unique(df.accession_number.values).shape